In [1]:
import numpy as np
from copy import deepcopy
import time
from pycromanager import Acquisition, Bridge, multi_d_acquisition_events

In [2]:
bridge = Bridge()
mmc = bridge.get_core()
mmStudio = bridge.get_studio()

C:\Users\labelfree\.conda\envs\acq\lib\site-packages\pycromanager\core.py:258: UserWarning: Version mistmatch between Java ZMQ server and Python client. 
Java ZMQ server version: 4.0.0
Python client expected version: 4.1.0
 To fix, update to BOTH latest pycromanager and latest micro-manager nightly build
  warnings.warn(


In [5]:
mm_pos_list = mmStudio.get_position_list_manager().get_position_list()
number_of_positions = mm_pos_list.get_number_of_positions()

xyz_position_list = [[mm_pos_list.get_position(i).get_x(),
                      mm_pos_list.get_position(i).get_y(),
                      mm_pos_list.get_position(i).get_z()] \
                    for i in range(number_of_positions)]

# offset = [mm_pos_list.get_position(i).get('PFSOffset').get1_d_position() for i in range(number_of_positions)]

xyz_position_list = np.around(xyz_position_list, decimals=3).astype('double')
# offset = np.around(offset, decimals=3).astype('double')

epi_positions = xyz_position_list[3:]
lf_epi_positions = xyz_position_list[:3]

# epi_positions = [xyz_position_list[0]]
# lf_epi_positions = [xyz_position_list[1]]

xyz_position_list

array([[ -5239.21 ,  14579.279,   4567.64 ],
       [ -6888.16 ,  14569.429,   4472.52 ],
       [ -8051.3  ,  14039.98 ,   4515.96 ],
       [-10455.869,  14042.96 ,   4492.7  ],
       [ -7875.56 ,  16009.189,   4481.68 ]])

In [9]:
epi_channel = {'group': 'Master Channel', 'config': 'Epi'}
epi_exposure = 100

lf_channels = [{'group': 'Master Channel', 'config': f'LF-State{i}'} for i in range(5)]
lf_exposure = 5

# relative z
z_start = -50
z_end = 50
z_step = 0.5
z_range = np.arange(z_start, z_end + z_step, z_step)

num_time_points  = 5
time_interval_s  = 20*60  # in seconds

In [10]:
events = []
num_epi_positions = np.shape(epi_positions)[0]

for t in range(num_time_points):
    
    # epi-only positions
    for p_idx, p in enumerate(epi_positions):
        for z_idx, z in enumerate(z_range+p[2]):
            events.append(
                {
                    'axes': {'time': t, 'position':p_idx, 'z':z_idx},
                    'min_start_time': t*time_interval_s,
                    'x': p[0],
                    'y': p[1],
                    'z': z,
                    'channel': epi_channel,
                    'exposure': epi_exposure,
                    'keep_shutter_open': False,
                })

    # epi and lf positions
    for p_idx, p in enumerate(lf_epi_positions):
        
        # epi channel, z-first
        for z_idx, z in enumerate(z_range+p[2]):
            events.append(
                {
                    'axes': {'time': t,'position':p_idx+num_epi_positions, 'z':z_idx},
                    'min_start_time': t*time_interval_s,
                    'x': p[0],
                    'y': p[1],
                    'z': z,
                    'channel': epi_channel,
                    'exposure': epi_exposure,
                    'keep_shutter_open': False
                })
            
        # lf channels, channel-first
        for z_idx, z in enumerate(z_range+p[2]):
            for ch in lf_channels:
                events.append(
                    {
                        'axes': {'time': t,'position':p_idx+num_epi_positions, 'z':z_idx},
                        'min_start_time': t*time_interval_s,
                        'x': p[0],
                        'y': p[1],
                        'z': z,
                        'channel': ch,
                        'exposure': lf_exposure,
                        'keep_shutter_open': False
                    })

In [11]:
mmc.set_auto_shutter(True)
# mmc.set_auto_focus_device('PFS')
mmc.set_config('Epi Channel', 'GFP EX466-40 EM525-50')

snap_live_manager = mmStudio.get_snap_live_manager()
if snap_live_manager.is_live_mode_on():
    snap_live_manager.set_live_mode_on(False)

with Acquisition(directory=r'D:\2022_02_16 Zebrafish v2', name='zebrafish_timeseries_t2') as acq:
    acq.acquire(events)